In [5]:
from jax.config import config
import jax
import numpy as np

devices = jax.devices()
print("\n".join([str(d) for d in devices]))
device0 = jax.devices()[0]

IpuDevice(id=0, num_tiles=1472, version=ipu2)


In [3]:
import tessellate_ipu

In [24]:
@jax.jit
def compute_fn(data):
    return jax.lax.abs(data)

data = np.random.rand(3, 5).astype(np.float32)
# Compilation & run
%time out = compute_fn(data)

CPU times: user 1.24 s, sys: 112 ms, total: 1.35 s
Wall time: 1.27 s


In [25]:
from tessellate_ipu import tile_put_sharded, tile_map

@jax.jit
def compute_fn(data):
    # Guide Poplar compilation as much as possible!
    t0 = tile_put_sharded(data, (0, 1, 3))
    return tile_map(jax.lax.abs_p, t0)

data = np.random.rand(3, 5).astype(np.float32)
# Compilation & run
%time out = compute_fn(data)

CPU times: user 1.33 s, sys: 54.8 ms, total: 1.38 s
Wall time: 1.28 s
